# Supermercato24 Out Of Stock Project

In [1]:
import pandas as pd
import numpy as np
import analitico.storage
import s24.categories
from datetime import datetime


### Load list of order details, joined orders, joined stores info

In [2]:
data_url = 'https://storage.googleapis.com/data.analitico.ai/data/s24/training/out-of-stock.csv'
data_filename = analitico.storage.download_file(data_url)
data_filename


'/tmp/data/s24/training/out-of-stock.csv'

In [3]:
#data_url = 'https://storage.googleapis.com/data.analitico.ai/data/s24/training/out-of-stock.csv'
#data_url = 'https://storage.googleapis.com/data.analitico.ai/data/s24/training/out-of-stock-top10000.csv'
#data_filename = analitico.storage.download_file(data_url)


datatypes = {
    'odt_ean': str,
    'odt_name': str,
    'odt_category_id': str,
    'odt_category_name': str,
    'odt_replaceable': int,
    "odt_variable_weight": int,
    "odt_price": float,
    "odt_price_per_type": float,
    "odt_surcharge_fixed": float,
    "odt_touched_at": str,
    "ord_id": str,
    "ord_gdo": str,
    "store_id": str,
    "store_name": str,
    "store_area": str,
    "odt_status": str    
}

data_filename = 'data/s24/training/out-of-stock.csv'
df = pd.read_csv(data_filename, dtype=datatypes)

print(df['odt_ean'].dtypes)

# drop incomplete orders
df = df.dropna(subset=['odt_category_id'])
df = df.dropna(subset=['odt_touched_at'])

# set index, binary status (purchased or not)
df.set_index(keys='odt_id', inplace=True, verify_integrity=True)
df['odt_touched_at'] = pd.to_datetime(df['odt_touched_at'])
df['dyn_purchased'] = np.where(df['odt_status'] == 'PURCHASED', 1, 0)

df = df.tail(500000)


object


In [4]:
df.columns

Index(['odt_ean', 'odt_name', 'odt_category_id', 'odt_category_name',
       'odt_replaceable', 'odt_variable_weight', 'odt_price',
       'odt_price_per_type', 'odt_surcharge_fixed', 'odt_touched_at', 'ord_id',
       'ord_gdo', 'store_id', 'store_name', 'store_area', 'odt_status',
       'dyn_purchased'],
      dtype='object')

In [5]:
df.tail(3)

,odt_ean,odt_name,odt_category_id,odt_category_name,odt_replaceable,odt_variable_weight,odt_price,odt_price_per_type,odt_surcharge_fixed,odt_touched_at,ord_id,ord_gdo,store_id,store_name,store_area,odt_status,dyn_purchased
odt_id,,,,,,,,,,,,,,,,,
15359627,8015100565825,Vernel Concentrato Aromaterapia Viola 1 Lt. - ...,1900,Ammorbidenti,2,0,1.85,0.01,0.0,2018-10-24 19:31:08,804611,8,3231,pam,MI4,PURCHASED,1
15359629,7615400173419,Mr Muscle Idraulgel 5in1,2307,Sgorgatori,2,0,3.34,0.01,0.0,2018-10-24 19:32:14,804611,8,3231,pam,MI4,REPLACED,0
15359631,8002295034915,"Winni's Lavatrice Aleppo e Verbena 1,25 l",1601,Detersivi per la Lavatrice,2,0,4.60,0.01,0.0,2018-10-24 19:33:01,804611,8,3231,pam,MI4,PURCHASED,1


## Create category labels at 3 levels

In [6]:
def get_category_id(row, level):
    try:
        if not pd.isnull(row['odt_category_id']):
            return str(s24.categories.s24_get_category_id(int(row['odt_category_id']), level))
    except TypeError:
        pass
    return None

df['dyn_main_category_id'] = df.apply(lambda row: get_category_id(row, 0), axis=1).astype('category') 
df['dyn_sub_category_id'] = df.apply(lambda row: get_category_id(row, 1), axis=1).astype('category')  
df['dyn_category_id'] = df.apply(lambda row: get_category_id(row, 2), axis=1).astype('category')


s24.categories - loaded in 3745 ms


## Out of stock by Product, Category, Store 

In [7]:
def aggregate_purchases(group, min_count=5):
    f1 = {'dyn_purchased': ['sum', 'count', 'mean'] }
    group = group.agg(f1)
    group = group.sort_values(('dyn_purchased', 'sum'), ascending=False)
    group = group[group[('dyn_purchased', 'sum')] > min_count] # minimum number of purchased
    return pd.DataFrame(group)

In [8]:
df2 = aggregate_purchases(df.groupby(['odt_ean', 'odt_name']))
df2.to_csv('/home/gionata/dumps/outofstock-by-ean.csv')
df2.head(10)

dyn_purchased  \
                                                                         sum   
odt_ean       odt_name                                                         
8017596071286 Delizie dal Sole Uova Grandi Allevamento a Terra           908   
8076809504720 Mulino Bianco Pan Bauletto Bianco                          778   
2070226000000 Zucchine                                                   766   
2070124000000 Banana                                                     750   
21306433      del Monte Banana                                           732   
2135545000000 Banana                                                     648   
8013355999143 Pavesi Gocciole Chocolate                                  573   
80000532      LEVISSIMA  Acqua Minerale Naturale Oligominerale           555   
8020141800002 Sant'Anna Naturale Sorgente Rebruant 1,5 Litri             536   
8017596007148 Bottega del Gusto Prosciutto Crudo                         516   

                                                                                
                                                               count      mean  
odt_ean       odt_name                                                          
8017596071286 Delizie dal Sole Uova Grandi Allevamento a Terra   944  0.961864  
8076809504720 Mulino Bianco Pan Bauletto Bianco                  839  0.927294  
2070226000000 Zucchine                                           791  0.968394  
2070124000000 Banana                                             760  0.986842  
21306433      del Monte Banana                                   741  0.987854  
2135545000000 Banana                                             650  0.996923  
8013355999143 Pavesi Gocciole Chocolate                          599  0.956594  
80000532      LEVISSIMA  Acqua Minerale Naturale Oligominerale   608  0.912829  
8020141800002 Sant'Anna Naturale Sorgente Rebruant 1,5 Litri     663  0.808446  
8017596007148 Bottega del Gusto Prosciutto Crudo                 516  1.000000

In [27]:
df5 = aggregate_purchases(df.groupby(['odt_ean']))
df5.to_csv('/home/gionata/dumps/outofstock-by-ean.csv')
#df5.head(10)

float(df5.loc[['8017596071286'], ('dyn_purchased', 'mean')])

0.961864406779661

In [9]:
df2 = aggregate_purchases(df.groupby(['odt_ean', 'odt_name']))
df3 = df2.sort_values(('dyn_purchased', 'mean'), ascending=True)
df2.head(100)

dyn_purchased  \
                                                                           sum   
odt_ean       odt_name                                                           
8017596071286 Delizie dal Sole Uova Grandi Allevamento a Terra             908   
8076809504720 Mulino Bianco Pan Bauletto Bianco                            778   
2070226000000 Zucchine                                                     766   
2070124000000 Banana                                                       750   
21306433      del Monte Banana                                             732   
2135545000000 Banana                                                       648   
8013355999143 Pavesi Gocciole Chocolate                                    573   
80000532      LEVISSIMA  Acqua Minerale Naturale Oligominerale             555   
8020141800002 Sant'Anna Naturale Sorgente Rebruant 1,5 Litri               536   
8017596007148 Bottega del Gusto Prosciutto Crudo                           516   
8017596059963 Land Bustone Mozzarella                                      500   
8076809517966 Mulino Bianco Pan Bauletto al Grano Duro                     475   
8054309130021 Pomodoro Ciliegino                                           473   
80582519      Sant'Anna Naturale Sorgente Rebruant 1,5 Litri               452   
21300608      Zucchina Scura                                               436   
80007920      San Benedetto Acqua Minerale  Benedicta Natural...           426   
8002004217554 Biraghi Gran Grattugiato Fresco                              412   
8017596017468 Pascoli Italiani Parmigiano Reggiano Grattugiat...           404   
8000430138665 Galbani Santa Lucia Tris Mozzarella                          400   
8008260001036 Foxy 4 Mega Rotoli di Carta Igienica                         395   
8076809524322 Mulino Bianco Plumcake Classico                              392   
2070249000000 Limoni                                                       391   
8001590640258 Philadelphia Original Classico Formaggio Fresco              389   
80369691      Vitasnella Acqua Oligominerale Naturale                      385   
80519676      Vallelata Mozzarella Fresca                                  383   
8017596009449 Blues Acqua Naturale                                         382   
8033874310021 Pomodoro Piccadilly                                          374   
80134091      Ferrarelle 1,5 Litri                                         369   
5029053021409 Scottex Tuttofare 2 Pz                                       367   
8017596030061 Foglia Verde Lattughino                                      366   
...                                                                        ...   
5449000133328 Coca-cola Zero Coca-cola  Zero Zuccheri Zero Ca...           260   
2003495110000 Coop Banane                                                  259   
8000430133455 Vallelata 8 Bocconcini di Mozzarella Fresca                  259   
8002580011355 Zymil Alta Digeribilità senza Lattosio senza Gr...           258   
2070207000000 Uva Red Globe                                                258   
8017596030115 Foglia Verde Cuori di Iceberg                                255   
8001590927366 Sottilette Classiche                                         253   
8017596014832 Delizie dal Sole  Ceci                                       251   
8000430070316 Galbani Galbanino Formaggio Dolce                            251   
8076802795019 Barilla Farina Tipo "00" di Grano Tenero                     251   
8076809540742 Barilla I Pesti Pesto alla Genovese senza Aglio              250   
8019514041722 Bennet Fresche Medie da Allevamento a Terra                  247   
21307591      Limone                                                       247   
8076809513340 Barilla I Pesti alla Genovese                                245   
2070123000000 Melanzana Tonda                                              244   
8017596005649 Bottega del Gusto Bresaola Punta 

In [149]:
df3 = aggregate_purchases(df.groupby(['dyn_category_id', 'odt_category_name']))
df3.to_csv('/home/gionata/dumps/outofstock-by-category_id.csv')
df3.head(100)

dyn_purchased                 
                                                         sum  count      mean
dyn_category_id odt_category_name                                            
261             Altre Verdure                          18072  20098  0.899194
254             Altra frutta                           12298  13985  0.879371
691             Acqua naturale                         10763  12787  0.841714
459             Pasta di semola corta                   8281   9188  0.901284
412             Merendine                               7597   8260  0.919734
300             Salumi da Banco                         7432   8186  0.907892
322             Latte a Lunga Conservazione             7181   8127  0.883598
265             Insalate Pulite e Lavate                6276   7364  0.852254
329             Uova                                    6259   7021  0.891468
260             Pomodori                                5881   6449  0.911924
252             Mele e Pere                             5466   6230  0.877368
352             Pane a Fette                            5392   6000  0.898667
295             Mozzarella Vaccina                      5025   5590  0.898927
654             Succhi a base di frutta                 4861   5299  0.917343
406             Biscotti frollini                       4744   5155  0.920272
3100            Carta igienica                          4734   5135  0.921908
206             Pollo                                   4708   5445  0.864646
251             Agrumi                                  4657   5213  0.893344
337             Yogurt intero ai gusti                  4450   4995  0.890891
351             Pane                                    4444   5764  0.770992
690             Acqua gassata ed effervescente          4369   4924  0.887287
454             Sughi pronti                            4364   4748  0.919124
541             Legumi e cereali in scatola             3942   4247  0.928185
281             Formaggi vari da Banco                  3691   4242  0.870108
285             Grattugiati                             3629   4045  0.897157
550             Patatine                                3609   4006  0.900899
268             Patate                                  3397   3804  0.893007
660             Cola                                    3237   3627  0.892473
702             Lager e Pilsner                         3189   3501  0.910883
264             Insalate e radicchi                     3086   3345  0.922571
...                                                      ...    ...       ...
601             Secondi di Pesce                        1680   1871  0.897916
282             Crescenze                               1632   1906  0.856243
409             Fette biscottate                        1632   1767  0.923599
334             Yogurt Benessere                        1618   1858  0.870829
387             Salse e sughi freschi                   1602   1777  0.901519
610             Filetti vari                            1598   1791  0.892239
312             Wurstel                                 1591   1700  0.935882
413             Caffè macinato                          1545   1691  0.913661
213             Carni Impanate                          1542   1893  0.814580
1003            Omogeneizzati                           1526   1672  0.912679
2200            Detersivi piatti a mano                 1523   1673  0.910341
392             Primi pronti - minestre                 1523   1757  0.866818
3102            Fazzoletti di carta                     1513   1703  0.888432
6047            Altre verdure                           1506   1653  0.911071
326             Panna da cucina                         1496   1656  0.903382
653             Succhi di frutta 100%                   1492   1671  0.892879
620             Pizza                                   1489   1704  0.873826
3103            Tovaglioli e tovaglie                   1476   1622  

In [150]:
df4 = aggregate_purchases(df.groupby(['store_id', 'store_name']))
df4.to_csv('/home/gionata/dumps/outofstock-by-store.csv')
df4.head(10)

dyn_purchased                 
                                     sum  count      mean
store_id store_name                                      
1295     mercatò                   19132  20674  0.925414
2769     famila superstore         16118  17896  0.900648
5032     eurospin                  15453  15915  0.970971
5030     eurospin                  15317  15992  0.957791
5057     conad superstore          13839  14759  0.937665
5031     eurospin                  11835  12375  0.956364
5033     eurospin                  11280  11800  0.955932
1199     bennet                    10910  12450  0.876305
5215     conad superstore           9752  10106  0.964971
5055     eurospin                   8764   9075  0.965730

In [152]:
def get_price(row):
    try:
        return float(row['odt_price']) if int(row['odt_variable_weight']) == 0 else float(row['odt_price_per_type'])
    except Exception as _:
        logger.error(str(row) + ' cannot calculate price')
        return None

def get_price_promo(self, row):
    try:
        return (row['dyn_price'] + row['odt_surcharge_fixed']) / row['dyn_price']
    except ZeroDivisionError:
        return 1

df['dyn_price'] = df.apply(lambda row: get_price(row), axis=1)
df['dyn_price_promo'] = df.apply(lambda row: get_price_promo(row), axis=1)
    

TypeError: ("get_price_promo() missing 1 required positional argument: 'row'", 'occurred at index 14091855')